# Training and A/B test for Conversion agents - <font color='blue'> Presence</font> of sale

In [16]:
import gym, recogym
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pandas as pd
import pickle as pkl
pd.options.mode.chained_assignment = None 
# from inspect import getsource
from recogym.evaluate_agent_sale import verify_agents_sale, display_metrics, verify_agents_sale_extended 
from tqdm import tqdm

# env_1_sale_args is a dictionary of default parameters (i.e. number of products)
from recogym import env_1_sale_args, Configuration
from recogym.agents.sale_agent import train_agents, train_timeagents
from recogym.envs.utils_sale import format_avg_result, avg_result, format_avg_result_extended, avg_result_extended


# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 0
env_1_sale_args['num_products'] = 10
env_1_sale_args['number_of_flips'] = 10 

num_products = env_1_sale_args['num_products']
print('Number of products =',num_products)
print('Number of flips =',env_1_sale_args['number_of_flips'])
nb_flips = env_1_sale_args['number_of_flips']

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 42
env_1_sale_args['mu_sale'] = False 

# env_1_sale_args['kappa'] = 0.5
print('Value of kappa =',env_1_sale_args['kappa'])

# Initialize the gym 
env = gym.make('reco-gym-sale-v1')
env.init_gym(env_1_sale_args)



Number of products = 10
Number of flips = 10
Value of kappa = 0.2


In [17]:
# Repository to save pickles
data_repo = 'data_conversion/'

## Train baseline agents

#### Settings

In [18]:
## Choose number of users for training and AB test
# Number of users for the training
# env_1_sale_args['num_users'] = 6 ##tochange !!
env_1_sale_args['num_users'] = 5000
num_users = env_1_sale_args['num_users']

# Number of users for the A/B test
# env_1_sale_args['num_users_AB'] = 7 ##tochange !!
env_1_sale_args['num_users_AB'] = 5000
num_users_AB = env_1_sale_args['num_users_AB']

# Choose user features
from recogym.agents.sale_agent import CountViewsClicksFeatureProvider, CountViewsFeatureProvider, ShareViewsClicksFeatureProvider, ShareViewsFeatureProvider
vc_feature = CountViewsClicksFeatureProvider(env.config)
v_feature = CountViewsFeatureProvider(env.config)
vc_share_feature = ShareViewsClicksFeatureProvider(env.config)
v_share_feature = ShareViewsFeatureProvider(env.config)
features = {'vc':vc_feature,
           'v':v_feature,
           'vc_share':vc_share_feature,
           'v_share':v_share_feature}
feature_name = 'v_share'
feature = features[feature_name]

#### <font color='red'> Number of A/B tests</font>

In [19]:
# # Choose number of A/B tests
num_AB_tests = 2 ##tochange !
# num_AB_tests = 25 ##tochange !

#### Logs

In [20]:
agents={}
logs={}

############## Random agent
name_agent = 'rand'+str(nb_flips)
from recogym.agents import RandomAgent, random_args
random_agent = RandomAgent(Configuration(random_args))
agents[name_agent] = random_agent


try:
    logs[name_agent] = pkl.load(open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','rb'))
    print('--- Logs loaded---')
except: 
    print("--- Generate logs ---")
    logs[name_agent] = deepcopy(env).generate_logs(num_users)
    print(data_repo + 'data' + str(num_users) + name_agent + '.pkl')
    pkl.dump(logs[name_agent], open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','wb'))

--- Logs loaded---


#### Training for the <font color='blue'> Presence</font> of sale

##### No weights

In [21]:
name_extension = 'pres'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('agents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_agents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,repo = data_repo)

--- Trained agents loaded --- 


In [22]:
try:
    for i in range(num_AB_tests):
        name_extension = 'pres'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discountpres_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        print(str(data_repo) +'res_'+agent_id+name+'.pkl')
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'pres'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

data_conversion/res_rand106_7_v_sharepres_nb0likelihood_saleclickprod_discountpres_full.pkl
data_conversion/res_rand106_7_v_sharepres_nb1likelihood_saleclickprod_discountpres_full.pkl
--- A/B test results loaded ---


In [23]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.881 (0.517) %,1 (0),16.03 (0.00) %,5 (2),27.99 (8.46) %
1,Click,0.854 (0.374) %,1 (0),19.02 (4.23) %,5 (0),25.00 (4.23) %
2,PDS,0.710 (0.444) %,1 (0),16.03 (0.00) %,7 (1),33.97 (8.45) %
3,PCS,0.653 (0.281) %,0 (0),10.09 (8.40) %,3 (4),22.02 (16.91) %
4,DPCS,0.895 (0.077) %,1 (1),16.03 (8.45) %,6 (8),25.00 (12.69) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.610 (0.089) %,4.29 (0.43) %,2.68 (0.72) %,3.26 (0.40) %,0.519 (0.024) pm,0.200 (0.064) pm,0.126 (0.014) pm
1,Click,0.712 (0.124) %,5.66 (0.92) %,4.14 (2.04) %,4.80 (1.19) %,0.795 (0.010) pm,0.231 (0.197) pm,0.126 (0.061) pm
2,PDS,0.676 (0.086) %,5.45 (0.51) %,4.27 (0.53) %,4.64 (0.50) %,0.732 (0.025) pm,0.160 (0.023) pm,0.110 (0.013) pm
3,PCS,0.759 (0.121) %,4.64 (1.85) %,3.16 (2.31) %,3.67 (2.28) %,0.682 (0.168) pm,0.230 (0.105) pm,0.153 (0.089) pm
4,DPCS,0.858 (0.040) %,4.96 (1.07) %,3.31 (1.18) %,4.23 (0.94) %,0.856 (0.223) pm,0.284 (0.006) pm,0.127 (0.028) pm


In [24]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &          Att CR &  Sales &               CR \\
\midrule
  Rand &  0.881 (0.517) \% &     1 (0) &  16.03 (0.00) \% &  5 (2) &   27.99 (8.46) \% \\
 Click &  0.854 (0.374) \% &     1 (0) &  19.02 (4.23) \% &  5 (0) &   25.00 (4.23) \% \\
   PDS &  0.710 (0.444) \% &     1 (0) &  16.03 (0.00) \% &  7 (1) &   33.97 (8.45) \% \\
   PCS &  0.653 (0.281) \% &     0 (0) &  10.09 (8.40) \% &  3 (4) &  22.02 (16.91) \% \\
  DPCS &  0.895 (0.077) \% &     1 (1) &  16.03 (8.45) \% &  6 (8) &  25.00 (12.69) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
  Rand &  0.610 (0.089) \% &  4.29 (0.43) \% &  2.68 (0.72) \% &  3.26 (0.40) \% &  0.519 (0.024) pm &  0.200 (0.064) pm &  0.126 (0.014) pm \\
 Click &  0.712 (0.124) \% &  5.66 (0.92) \% &  4.14 (2.04) \% & 

##### Weights

In [25]:
name_extension = 'presweights'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('agents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_agents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,weights = True, repo = data_repo)

--- Trained agents loaded --- 


In [26]:
try:
    for i in range(num_AB_tests):
        name_extension = 'presweights'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discountpresweights_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'presweights'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [27]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.297 (0.034) %,0 (1),10.09 (8.40) %,6 (8),19.06 (21.09) %
1,Click,0.420 (0.133) %,0 (1),10.09 (8.40) %,8 (6),25.00 (4.23) %
2,PDS,0.611 (0.080) %,0 (0),13.04 (4.22) %,9 (0),39.94 (0.00) %
3,PCS,0.501 (0.386) %,1 (1),13.08 (12.63) %,7 (5),27.98 (16.91) %
4,DPCS,0.820 (0.235) %,1 (0),10.06 (0.00) %,9 (0),27.99 (0.00) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.576 (0.037) %,4.63 (0.53) %,3.23 (0.12) %,3.61 (0.50) %,0.531 (0.027) pm,0.159 (0.037) pm,0.117 (0.004) pm
1,Click,0.530 (0.035) %,5.72 (1.61) %,4.77 (1.56) %,5.08 (2.26) %,0.601 (0.131) pm,0.100 (0.001) pm,0.070 (0.073) pm
2,PDS,0.556 (0.011) %,5.17 (1.38) %,3.85 (1.48) %,4.41 (1.98) %,0.574 (0.143) pm,0.146 (0.014) pm,0.086 (0.068) pm
3,PCS,0.685 (0.153) %,5.50 (1.30) %,4.23 (2.26) %,4.69 (1.65) %,0.734 (0.010) pm,0.189 (0.170) pm,0.117 (0.072) pm
4,DPCS,0.654 (0.042) %,6.32 (0.57) %,5.62 (0.80) %,5.96 (0.53) %,0.825 (0.021) pm,0.094 (0.037) pm,0.048 (0.002) pm


In [28]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &           Att CR &  Sales &               CR \\
\midrule
  Rand &  0.297 (0.034) \% &     0 (1) &   10.09 (8.40) \% &  6 (8) &  19.06 (21.09) \% \\
 Click &  0.420 (0.133) \% &     0 (1) &   10.09 (8.40) \% &  8 (6) &   25.00 (4.23) \% \\
   PDS &  0.611 (0.080) \% &     0 (0) &   13.04 (4.22) \% &  9 (0) &   39.94 (0.00) \% \\
   PCS &  0.501 (0.386) \% &     1 (1) &  13.08 (12.63) \% &  7 (5) &  27.98 (16.91) \% \\
  DPCS &  0.820 (0.235) \% &     1 (0) &   10.06 (0.00) \% &  9 (0) &   27.99 (0.00) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
  Rand &  0.576 (0.037) \% &  4.63 (0.53) \% &  3.23 (0.12) \% &  3.61 (0.50) \% &  0.531 (0.027) pm &  0.159 (0.037) pm &  0.117 (0.004) pm \\
 Click &  0.530 (0.035) \% &  5.72 (1.61) \% &  4.77 (1.56)